# Data Cleaning, Preprocessing and Analysis

In this file, we will Clean and Analyse our dataset. 

For Cleaning, we use Percentiles and remove outliers. We then preprocess the data and perform certain Analysis such as Granger Causality, Johanson's Cointigration test and ADF Test. These tests are done to prove as to why we are using the Vector Autoregressive Model for our solution

In [0]:
import numpy as np
import pandas as pd
from pyspark.sql import functions as F

from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import DoubleType
from pyspark.sql.types import *
import time
import random

import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.stats.stattools import durbin_watson
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error
import math
from statsmodels.tsa.stattools import acf
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

<h3> Import Data

In [0]:
def import_data(filepaths):
  """
  Import Data from list of filepaths and concatanate 
  """
  df = spark.read.csv(filepaths[0],header=True)
  for path in filepaths[1:]:
    df = df.unionByName(spark.read.csv(path,header=True))
  return df


# Import Data
taxi = import_data(["/mnt/pirates-of-the-carribbytes/taxi_data/taxi_2019-01.csv",
                    "/mnt/pirates-of-the-carribbytes/taxi_data/taxi_2019-02.csv",
                    "/mnt/pirates-of-the-carribbytes/taxi_data/taxi_2019-03.csv"])


# Import Zone data and join with our dataset. Remove datapoints from EWR and Staten Island
zones = spark.read.csv("/mnt/pirates-of-the-carribbytes/taxi_data/zone_lookup.csv",header=True).drop("Zone","service_zone")
taxi = taxi.join(zones,taxi.PULocationID == zones.LocationID)\
                 .withColumnRenamed("Borough","Division")\
                 .filter((F.col('Division') != 'EWR') & (F.col('Division') != 'Staten Island'))

# Filter to ensure no outlier years are included (we found a timestamp of 2088) and remove unnecessary columns
taxi = taxi.filter((F.unix_timestamp(taxi.tpep_pickup_datetime)>=1546300800) & (F.unix_timestamp(taxi.tpep_pickup_datetime)<=1554073200))\
           .drop('VendorID', 'RatecodeID', 'store_and_fwd_flag','DOLocationID','fare_amount','extra',"LocationID","PULocationID",
                 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'payment_type')



divs = ["Manhattan", "Queens", "Bronx","Brooklyn", "Unknown"]
taxi.count()

Out[4]: 22511036

We have removed some attributes such as **'VendorID', 'RatecodeID', 'store_and_fwd_flag','DOLocationID', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge'** \- and work with the remaining data. The Zones data has been joined to our dataset

<h1> Data Preprocessing and Cleaning

In this section, we will clean our data and remove all outliers. We can also create new attributes that may help in the final output.

In [0]:
taxi.show(10)

+--------------------+---------------------+---------------+-------------+------------+---------+
tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|total_amount| Division|
+--------------------+---------------------+---------------+-------------+------------+---------+
 2019-01-01 00:46:40| 2019-01-01 00:53:20| 1| 1.50| 9.95|Manhattan|
 2019-01-01 00:59:47| 2019-01-01 01:18:59| 1| 2.60| 16.3|Manhattan|
 2019-01-01 00:21:28| 2019-01-01 00:28:37| 1| 1.30| 9.05|Manhattan|
 2019-01-01 00:32:01| 2019-01-01 00:45:39| 1| 3.70| 18.5|Manhattan|
 2019-01-01 00:57:32| 2019-01-01 01:09:32| 2| 2.10| 13|Manhattan|
 2019-01-01 00:24:04| 2019-01-01 00:47:06| 2| 2.80| 19.55|Manhattan|
 2019-01-01 00:21:59| 2019-01-01 00:28:24| 1| .70| 8.5|Manhattan|
 2019-01-01 00:45:21| 2019-01-01 01:31:05| 1| 8.70| 42.95|Manhattan|
 2019-01-01 00:43:19| 2019-01-01 01:07:42| 1| 6.30| 28.5|Manhattan|
 2019-01-01 00:58:24| 2019-01-01 01:15:18| 1| 2.70| 15.3|Manhattan|
+--------------------+---------------------+---------------+-------------+------------+---------+
only showing top 10 rows

<h4> Construct New features such as Trip Duration (and Speed?)

- Since we have the Trip start & end time, we can compute the Duration by (Dropoff Time - Pickup Time)/60 --- To get the duration in minutes. 
- Given the Distance travelled, we can also compute the Speed of the journey (Speed = (Distance Travelled/Duration)\*60) ---- Speed in miles/hour


For further computation, we can convert the pickup and drop datetime into UNIX timestamps, and drop the columns that are not required in our case

In [0]:
#Convert to Unix timestamp and drop columns that aren't required
taxi_unix = taxi.select("*",
                        F.unix_timestamp(taxi.tpep_pickup_datetime).alias('Pickup_Unix'),
                        F.unix_timestamp(taxi.tpep_dropoff_datetime).alias('Dropoff_Unix'))
taxi_unix.show(2)

+--------------------+---------------------+---------------+-------------+------------+---------+-----------+------------+
tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|total_amount| Division|Pickup_Unix|Dropoff_Unix|
+--------------------+---------------------+---------------+-------------+------------+---------+-----------+------------+
 2019-01-01 00:46:40| 2019-01-01 00:53:20| 1| 1.50| 9.95|Manhattan| 1546303600| 1546304000|
 2019-01-01 00:59:47| 2019-01-01 01:18:59| 1| 2.60| 16.3|Manhattan| 1546304387| 1546305539|
+--------------------+---------------------+---------------+-------------+------------+---------+-----------+------------+
only showing top 2 rows

In [0]:
# Create the Duration and Speed Column
taxi_ds = taxi_unix.withColumn("Duration",
                               F.round((taxi_unix.Dropoff_Unix-taxi_unix.Pickup_Unix)/60,2))\
                   .withColumn("Speed",
                               F.round(60*(taxi_unix.trip_distance/((taxi_unix.Dropoff_Unix-taxi_unix.Pickup_Unix)/60)),2))\
                   .drop("Dropoff_Unix","tpep_pickup_datetime","tpep_dropoff_datetime")
taxi_ds.show(5)

+---------------+-------------+------------+---------+-----------+--------+-----+
passenger_count|trip_distance|total_amount| Division|Pickup_Unix|Duration|Speed|
+---------------+-------------+------------+---------+-----------+--------+-----+
 1| 1.50| 9.95|Manhattan| 1546303600| 6.67| 13.5|
 1| 2.60| 16.3|Manhattan| 1546304387| 19.2| 8.13|
 1| 1.30| 9.05|Manhattan| 1546302088| 7.15|10.91|
 1| 3.70| 18.5|Manhattan| 1546302721| 13.63|16.28|
 2| 2.10| 13|Manhattan| 1546304252| 12.0| 10.5|
+---------------+-------------+------------+---------+-----------+--------+-----+
only showing top 5 rows

In [0]:
taxi_ds.show(10)

+---------------+-------------+------------+---------+-----------+--------+-----+
passenger_count|trip_distance|total_amount| Division|Pickup_Unix|Duration|Speed|
+---------------+-------------+------------+---------+-----------+--------+-----+
 1| 1.50| 9.95|Manhattan| 1546303600| 6.67| 13.5|
 1| 2.60| 16.3|Manhattan| 1546304387| 19.2| 8.13|
 1| 1.30| 9.05|Manhattan| 1546302088| 7.15|10.91|
 1| 3.70| 18.5|Manhattan| 1546302721| 13.63|16.28|
 2| 2.10| 13|Manhattan| 1546304252| 12.0| 10.5|
 2| 2.80| 19.55|Manhattan| 1546302244| 23.03| 7.29|
 1| .70| 8.5|Manhattan| 1546302119| 6.42| 6.55|
 1| 8.70| 42.95|Manhattan| 1546303521| 45.73|11.41|
 1| 6.30| 28.5|Manhattan| 1546303399| 24.38| 15.5|
 1| 2.70| 15.3|Manhattan| 1546304304| 16.9| 9.59|
+---------------+-------------+------------+---------+-----------+--------+-----+
only showing top 10 rows

In [0]:
def calculate_percentile(df):
  """
  Function to calculate percentiles by division
  https://stackoverflow.com/questions/57424211/how-compute-the-percentile-in-pyspark-dataframe-for-each-key
  """
  for feature in df.columns:
    if feature != 'Division' and feature != 'Pickup_Unix':
      print("Percentiles for {}".format(feature))
      df.groupby('Division').agg(F.expr('percentile('+feature+', array(0.25))')[0].alias('25%'),
                                 F.expr('percentile('+feature+', array(0.50))')[0].alias('50%'),
                                 F.expr('percentile('+feature+', array(0.75))')[0].alias('75%'),
                                 F.expr('percentile('+feature+', array(0.90))')[0].alias('90%'),
                                 F.expr('percentile('+feature+', array(0.95))')[0].alias('95%'),
                                 F.expr('percentile('+feature+', array(0.96))')[0].alias('96%'),
                                 F.expr('percentile('+feature+', array(0.98))')[0].alias('98%'),
                                 F.expr('percentile('+feature+', array(0.99))')[0].alias('99%'),
                                 F.expr('percentile('+feature+', array(0.995))')[0].alias('99.5%'),
                                 F.expr('percentile('+feature+', array(0.999))')[0].alias('99.9%'),
                                 F.expr('percentile('+feature+', array(1.00))')[0].alias('100%'))\
         .show()
      
calculate_percentile(taxi_ds)

Percentiles for passenger_count
+---------+---+---+---+---+---+---+---+---+-----+-----+----+
 Division|25%|50%|75%|90%|95%|96%|98%|99%|99.5%|99.9%|100%|
+---------+---+---+---+---+---+---+---+---+-----+-----+----+
 Queens|1.0|1.0|2.0|3.0|5.0|5.0|6.0|6.0| 6.0| 6.0| 9.0|
 Unknown|1.0|1.0|1.0|2.0|4.0|4.0|5.0|6.0| 6.0| 6.0| 9.0|
 Brooklyn|1.0|1.0|2.0|3.0|5.0|5.0|6.0|6.0| 6.0| 6.0| 7.0|
Manhattan|1.0|1.0|2.0|3.0|5.0|5.0|6.0|6.0| 6.0| 6.0| 9.0|
 Bronx|1.0|1.0|1.0|2.0|5.0|5.0|6.0|6.0| 6.0| 6.0| 6.0|
+---------+---+---+---+---+---+---+---+---+-----+-----+----+

Percentiles for trip_distance
+---------+----+-----+-----+-----+------------------+-----+------------------+------------------+-----------------+-----------------+------+
 Division| 25%| 50%| 75%| 90%| 95%| 96%| 98%| 99%| 99.5%| 99.9%| 100%|
+---------+----+-----+-----+-----+------------------+-----+------------------+------------------+-----------------+-----------------+------+
 Queens|7.23|10.51|17.09| 19.2| 20.8|21.22| 22.92| 26.6|29.33905000000028| 42.3|237.17|
 Unknown| 0.7| 1.34| 2.5| 5.5| 9.6| 10.8| 16.82| 18.7| 20.6| 28.9956799999997|214.01|
 Brooklyn|1.26| 2.9| 5.9| 10.3| 14.05| 15.2| 18.5| 21.2| 23.71| 28.67|142.88|
Manhattan| 0.9| 1.5| 2.55| 4.6| 6.87| 7.79| 10.72| 15.63| 17.71| 21.45| 831.8|
 Bronx|1.68| 4.66| 10.3|15.89|19.088499999999986| 19.9|22.135400000000008|25.187699999999968| 27.59|35.53677999999985|209.38|
+---------+----+-----+-----+-----+------------------+-----+------------------+------------------+-----------------+-----------------+------+

Percentiles for total_amount
+---------+-----+------+-----+------------------+----------------+-----+-----------------+-----+-----------------+------------------+----------+
 Division| 25%| 50%| 75%| 90%| 95%| 96%| 98%| 99%| 99.5%| 99.9%| 100%|
+---------+-----+------+-----+------------------+----------------+-----+-----------------+-----+-----------------+------------------+----------+
 Queens|30.36|46.175| 61.3| 73.2| 75.67|76.32| 80.17|93.32| 132.67|213.32543000000294| 671124.94|
 Unknown| 8.3| 11.76| 18.3| 39.95|65.6504999999999|70.27|86.12519999999901|126.3| 170.8|284.30641999999995| 90000.0|
 Brooklyn| 9.36| 15.96| 26.3| 39.8| 50.64|53.78| 63.36|71.32| 78.3| 106.56| 3345.3|
Manhattan| 9.95| 12.96|17.76| 24.96| 32.25|35.31| 48.06|63.06| 73.26| 98.67|1084772.17|
 Bronx| 10.8| 19.8|37.63|54.484000000000094| 63.56|66.34| 75.56|84.56|96.78390000000029|165.42777999999672| 680.3|
+---------+-----+------+-----+------------------+----------------+-----+-----------------+-----+-----------------+------------------+----------+

Percentiles for Duration
+---------+-----+-----+-----+-----------------+------------------+-----------------+------------------+----------------+------------------+------------------+--------+
 Division| 25%| 50%| 75%| 90%| 95%| 96%| 98%| 99%| 99.5%| 99.9%| 100%|
+---------+-----+-----+-----+-----------------+------------------+-----------------+------------------+----------------+------------------+------------------+--------+
 Queens|18.62|28.92|40.08| 51.8| 60.03| 62.68| 71.4| 82.52|133.87435000000755| 1419.805430000003| 8590.33|
 Unknown| 4.83| 9.2|15.67| 24.38| 31.4| 33.97| 42.33| 51.67| 63.26209999999963| 1402.98642| 7190.9|
 Brooklyn| 7.42|14.25|24.47| 37.63|47.116999999999535| 50.12|59.256799999999345| 68.57| 79.58|1425.0975499999972| 1439.98|
Manhattan| 6.2|10.15|16.05| 23.55| 29.37| 31.35| 38.35| 47.63| 61.5| 1426.45|43648.02|
 Bronx| 9.63|19.13|32.97|48.17700000000004|58.696999999999974|62.17080000000002| 72.22540000000001|82.3230999999999| 94.77080000000016|1419.9104499999992| 1439.42|
+---------+-----+-----+-----+-----------------+------------------+-----------------+------------------+----------------+------------------+------------------+--------+

Percentiles for Speed
+---------+-----+-----+-----+-----+-----+-----------------+-----------------+------------------+------------------+------------------+--------+
 Division| 25%| 50%| 75%| 90%| 95%| 96%| 98%| 99%| 99.5%| 99.9%| 100%|
+

<h4> Trip Duration

**Observation:**
- Values seem fine until 90th percentile.
- 100th Percentile value is very large for a trip duration. 
- Values until 99th percentile seem okay.
- The outlier is just between 99th and 100th percentile.

We can look further into this if needed. 
But from the NYC Taxi & Limousine Commission regulations, drivers must limit themselves to 12 working hours/day (https://abc7ny.com/cab-drivers-taxi-hours-new-york-city/1353642/). 

So we can consider the thresholds to be 
- **minimum : 1 minute** 
- **maximum : 720 minutes** (12hrs\*60min)

**Observation:**

In [0]:
# Filtering based on 1 =< Duration =< 720
filtered_duration_df = taxi_ds.where((taxi_ds.Duration>=1) & (taxi_ds.Duration<=720))

<h4> Speed

**Observation:**
- From analyzing the percentile values, we can see that 99.9% of the values are below 47.6

So we can consider the thresholds to be 
- **minimum : 1 mile/hr** 
- **maximum : 72miles/hr**

In [0]:
# Filtering based on 1 <= Speed <= 720
filtered_speed_df = filtered_duration_df.where((filtered_duration_df.Speed>0) & (filtered_duration_df.Speed<=72))

<h4> Trip Distance

A good cutoff based on  distance seems to be 42 miles. We can remove the remaining data

In [0]:
# Filtering based on 0 < Trip Distance <= 10
filtered_distance_df = filtered_speed_df.where((filtered_speed_df['trip_distance']>0) & (filtered_speed_df['trip_distance']<=42))

<h4> Total Fare Amount

**Observation:**
- There are some negative fare values which does not make any logical sense.

The minimum fare amount according to the New York TLC is $2.50 (https://www1.nyc.gov/site/tlc/passengers/taxi-fare.page), so we can remove all datapoints with values lower than that.

In [0]:
# Filtering based on 0 < Fair Amount <= 10
filtered_fare_df = filtered_distance_df.where((filtered_distance_df['total_amount']>=2.5))

<h4> Passenger Count

From general analysis, we saw that there are some NULL values in this column. We will filter out the rows with these NULL passenger_count values and with passenger count of 0

In [0]:
#Check number of NULL values
print("Number of NULL values in passenger_count attribute: ",filtered_fare_df.filter((F.isnull(filtered_fare_df['passenger_count'])==True) | (filtered_fare_df['passenger_count']==0)).count())

#Filter out rows with NULL passenger count
filtered_pass_df = filtered_fare_df.filter((F.isnull(filtered_fare_df['passenger_count'])==False) & (filtered_fare_df['passenger_count']>0))

Number of NULL values in passenger_count attribute: 269393

###Binning

In [0]:
t = filtered_pass_df.withColumn("BIN",((filtered_pass_df.Pickup_Unix - 1546300800)/1800).cast("int")).withColumn("COUNT",F.lit(1))



taxi_binned = t.groupBy(["BIN","Division"])\
                .agg(F.mean("Duration"),F.mean("Speed"),F.mean("trip_distance"),F.mean("total_amount"),F.sum("COUNT"))\
                .sort("BIN")\
                .drop("BIN")
taxi_binned.show(10)

+---------+------------------+------------------+------------------+------------------+----------+
 Division| avg(Duration)| avg(Speed)|avg(trip_distance)| avg(total_amount)|sum(COUNT)|
+---------+------------------+------------------+------------------+------------------+----------+
Manhattan|16.365208706786145|12.075956466069147|3.1881049935979533|17.146140845070992| 3905|
 Brooklyn|18.468101265822785| 14.90594936708861| 4.774683544303798|21.909873417721503| 79|
 Queens|24.794840425531905| 29.34553191489363| 12.44515957446809|44.263510638297916| 188|
 Bronx| 22.53857142857143| 22.41857142857143| 9.822857142857144|32.942142857142855| 14|
 Unknown|14.594057971014491|12.393188405797098|2.9457971014492754|15.993478260869558| 69|
 Bronx|18.461666666666662|17.112222222222226| 6.16111111111111|22.578888888888898| 18|
 Queens|23.467619047619053|24.124074074074073|10.151164021164025|37.093703703703795| 189|
 Unknown| 15.4345|12.787000000000004|3.1650999999999994|16.494599999999988| 100|
 Brooklyn|21.235891891891892|14.703621621621615| 5.394918918918919|23.786216216216275| 185|
Manhattan| 16.48158009444245|11.646198692335663| 3.161905194333451|17.302742462768787| 5506|
+---------+------------------+------------------+------------------+------------------+----------+
only showing top 10 rows

<h3> Remove Outliers

In [0]:
def remove_outliers(df):
  """
  This function preprocesses our data,
  removes unnecessary outliers and
  creates 10 min time bins
  """
  
  # Create Pickup and Dropoff UNIX time from datetime feature
  df = df.select("*", F.unix_timestamp(df.tpep_pickup_datetime).alias('Pickup_Unix'),
                      F.unix_timestamp(df.tpep_dropoff_datetime).alias('Dropoff_Unix'))
  
  #Changing the datatype in numerical columns to double
  tcols = ['passenger_count', 'trip_distance', 'total_amount']
  for c in tcols:
    # add condition for the cols to be type cast
    df = df.withColumn(c, df[c].cast('double'))
  
  # Create Duration and Speed columns, remove Dropoff_Unix, datetime of pickup and dropoff
  df = df.withColumn("Duration", F.round((df['Dropoff_Unix'] - df['Pickup_Unix'])/60,2))\
                        .withColumn("Speed", F.round(60*(df['trip_distance'] / ((df['Dropoff_Unix']-df['Pickup_Unix'])/60)),2))\
                        .drop("Dropoff_Unix","tpep_dropoff_datetime","tpep_pickup_datetime")
  
  
  #Filter Outliers, Add Time Bin
  df = df.where((df['Duration']>=1) & (df['Duration']<=720))\
         .where((df['Speed']>0) & (df['Speed']<=72))\
         .where((df['trip_distance']>0) & (df['trip_distance']<=42))\
         .where((df['total_amount']>=2.5) & (df['total_amount']<=150))\
         .filter((F.isnull(df['passenger_count'])==False) & (df['passenger_count']>0))\
         .withColumn("BIN",((df.Pickup_Unix - 1546300800)/1800).cast("int")).withColumn("COUNT",F.lit(1))

  # Group By Time Bins and Divisions and get final dataset
  df = df.groupBy(["BIN","Division"])\
         .agg(F.mean("Duration"),F.mean("Speed"),F.mean("trip_distance"),F.mean("total_amount"),F.sum("COUNT"))\
         .sort("BIN")\
         .drop("BIN")

  return df


taxi_clean = remove_outliers(taxi).cache()
taxi_clean.show(10)

+---------+------------------+------------------+------------------+------------------+----------+
 Division| avg(Duration)| avg(Speed)|avg(trip_distance)| avg(total_amount)|sum(COUNT)|
+---------+------------------+------------------+------------------+------------------+----------+
 Queens|23.323599999999995| 28.03980000000001|11.600000000000007| 41.24070000000006| 200|
Manhattan|13.872607404474332| 11.39126905563253| 2.614274401108694|14.997301524451268| 5051|
 Bronx|21.441333333333333|21.410666666666668| 9.217333333333334| 31.23266666666667| 15|
 Brooklyn|16.306881720430113|14.075161290322583| 4.152258064516129| 19.60913978494622| 93|
 Unknown|12.685227272727277| 11.43840909090909| 2.4475|15.218295454545443| 88|
 Unknown|13.614958677685948|12.320165289256202|2.7375206611570233|15.351900826446256| 121|
 Bronx|18.461666666666662|17.112222222222226| 6.16111111111111|22.578888888888898| 18|
Manhattan|14.066465578424395| 11.03940667139814| 2.618150461320077|15.181650816182676| 7045|
 Queens| 21.41492890995261|22.801611374407575| 9.161611374407586| 34.07146919431289| 211|
 Brooklyn| 19.45227053140097| 14.14574879227053| 4.891884057971016| 22.03106280193242| 207|
+---------+------------------+------------------+------------------+------------------+----------+
only showing top 10 rows

In [0]:
divs = ["Manhattan", "Queens", "Brooklyn", "Bronx", "Unknown"]

for x in divs:
  temp = t.withColumn("Division", F.when(F.col("Division")==x, 1).otherwise(0))
  locals()["df_" + x] = temp.filter("Division > 0")

divs = ["Manhattan", "Queens", "Brooklyn", "Bronx", "Unknown"]
dfs = [df_Manhattan, df_Queens, df_Brooklyn, df_Bronx, df_Unknown]

for i in range(0,5):
  #groupBy and .filter("Division == 'Bronx'")\
  locals()["s" + divs[i]] = dfs[i].groupBy("BIN")\
                .agg(F.mean("Duration"),F.mean("Speed"),F.mean("trip_distance"),F.mean("total_amount"),F.sum("COUNT"))\
        .sort("BIN")\
        .drop("BIN")

##Check Granger Causality

In [0]:
from statsmodels.tsa.stattools import grangercausalitytests

maxlag=12
test = 'ssr_chi2test'
# [10 min, hourly ,12 hours, 24 hours, Weekly]
# [1,6,72,144,1008]
# sdf.select(sdf.columns[i]).collect()

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    

    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data.select(r,c).collect(), maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(sManhattan, variables = sManhattan.columns) 

Out[21]:

,avg(Duration)_x,avg(Speed)_x,avg(trip_distance)_x,avg(total_amount)_x,sum(COUNT)_x
avg(Duration)_y,1.0,0.0,0.0,0.0,0.0
avg(Speed)_y,0.0,1.0,0.0,0.0,0.0
avg(trip_distance)_y,0.0,0.0,1.0,0.0,0.0
avg(total_amount)_y,0.0,0.0,0.0,1.0,0.0
sum(COUNT)_y,0.0,0.0,0.0,0.0,1.0


##Perform Johanson's Cointegration Test and Report Summary

In [0]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen

def adjust(val, length= 6): return str(val).ljust(length)

def cointegration_test(df, alpha=0.05): 
    """Perform Johanson's Cointegration Test and Report Summary"""
    df = df.toPandas()
    out = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = out.lr1
    cvts = out.cvt[:, d[str(1-alpha)]]
    

    # Summary
    print('Name   ::  Test Stat > C(95%)    =>   Signif  \n', '--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), ':: ', adjust(round(trace,2), 9), ">", adjust(cvt, 8), ' =>  ' , trace > cvt)
        
        
cointegration_test(sManhattan)

Name :: Test Stat > C(95%) => Signif 
 ----------------------------------------
avg(Duration) :: 1781.89 > 60.0627 => True
avg(Speed) :: 1054.56 > 40.1749 => True
avg(trip_distance) :: 444.1 > 24.2761 => True
avg(total_amount) :: 141.07 > 12.3212 => True
sum(COUNT) :: 0.23 > 4.1296 => False

<h3> Train & Test Split

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import DoubleType

nobs = round(0.2*(sManhattan.count()))
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *
schema = StructType([
        StructField("avg(Duration)", DoubleType(), True),
        StructField("avg(Speed)", DoubleType(), True),
        StructField("avg(trip_distance)", DoubleType(), True),
        StructField("avg(total_amount)", DoubleType(), True),
        StructField("sum(COUNT)", DoubleType(), True)
    ])

@pandas_udf(schema,PandasUDFType.GROUPED_MAP)
def trainSplit(df):
  nobs = round(0.05*len(df))
  df_train = df[0:-nobs]
  return df_train

@pandas_udf(schema,PandasUDFType.GROUPED_MAP)
def testSplit(df):
  nobs = round(0.05*len(df))
  df_test = df[-nobs:]
  return df_test

sdf_train = sManhattan.groupby().apply(trainSplit)
sdf_test = sManhattan.groupby().apply(testSplit)

<h3> Augmented Dickey-Fuller Test (ADF Test) for Stationarity

In [0]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 

    if p_value <= signif:
        return [name," is stationary"]
    else:
        return [name," is non-stationary"]
      
      
      
      
schemaArr = StructType([
        StructField("Stationarity", StringType(), True),
        StructField("Column", StringType(), True)
    ])
@pandas_udf(schemaArr,PandasUDFType.GROUPED_MAP)
def adFullerStationarity_test(df_train):
  df_af = pd.DataFrame()
  for name, column in df_train.iteritems():
      x= adfuller_test(column, name=column.name)
      df_af = df_af.append({"Column":x[0],"Stationarity":x[1]},ignore_index=True )
  return df_af

sdf_train.groupby().apply(adFullerStationarity_test).show()

+--------------+------------------+
 Stationarity| Column|
+--------------+------------------+
 is stationary| avg(Duration)|
 is stationary| avg(Speed)|
 is stationary|avg(trip_distance)|
 is stationary| avg(total_amount)|
 is stationary| sum(COUNT)|
+--------------+------------------+

**As per the above analysis and tests, we conclude that we can use the Vector Autoregressor Model for our time series prediction**